In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
from hazm import word_tokenize

In [2]:
def find_elements_by_attribute(root, attribute_name, attribute_value):
    return root.findall(".//*[@{}='{}']".format(attribute_name, attribute_value))

In [3]:
def list_all_folders(directory):
    folders = []
    for item in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, item)):
            folders.append(item)
    return folders

In [4]:
def wrong_correct_sentence(root,begin,end,correct_word):
    wrong_sentence , correct_sentence="",""
    for element in root.findall("{http:///de/tudarmstadt/ukp/dkpro/core/api/segmentation/type.ecore}Sentence"):
        if int(element.get("begin"))<= begin and int(element.get("end"))>=end:
            wrong_sentence=root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[int(element.get("begin")):int(element.get("end"))] 
            s_begin=int(element.get("begin"))
            s_end=int(element.get("end"))
            if correct_word==None:
                correct_word=" "
            correct_sentence=root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[s_begin:begin]+correct_word+root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[end:s_end] 

    return wrong_sentence, correct_sentence


In [5]:
data={
    'file_name':[],
    'USER_NUMBER':[],
    'begin':[],
    'end':[],
    'wrong_word':[],
    'correct_word':[],
    'wrong_sentence':[],
    'correct_sentence':[],
    'process':[],
    'main_category':[],
    'sub_category_1':[],
    'sub_category_2':[]

}
df = pd.DataFrame(data)
df.to_csv('data.csv')

In [6]:
errors={
    "Main signs":1,
    "Diacritic signs":2,
    "Form":3,
    "Punctuation":4,
    None:5
}
sub_category_1={
    None:0,
    "None":0,
    "Boundary":1,
    "Consonants":2,
    "Vowels":3,
    "Tashdid":4,
    "Madd":5,
    "Tanwin":6,
    "Alif Lam":7,
    "Place":8,
    "Dot":9
    
    
}

In [7]:
def extract_data_from_XMI(path,USER_NUMBER):
    data_temp={
    'file_name':[],
    'USER_NUMBER':[],
    'begin':[],
    'end':[],
    'wrong_word':[],
    'correct_word':[],
    'wrong_sentence':[],
    'correct_sentence':[],
    'process':[],
    'main_category':[],
    'sub_category_1':[],
    'sub_category_2':[]
}
    tree = ET.parse(path)
    root=tree.getroot()
    if len(root.findall('{http:///webanno/custom.ecore}Orthography'))!=0 :
        # print(root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString"))
        for element in root.findall('{http:///webanno/custom.ecore}Orthography'):
            data_temp['USER_NUMBER'].append(USER_NUMBER)
            begin=element.get("begin")
            end=element.get("end")
            data_temp['file_name'].append(root.findall("{http:///de/tudarmstadt/ukp/dkpro/core/api/metadata/type.ecore}DocumentMetaData")[0].get("documentTitle"))
            data_temp['begin'].append(begin)
            data_temp['end'].append(end)
            data_temp['correct_word'].append(element.get("correction"))
            data_temp['main_category'].append(element.get("mainCategory"))
            data_temp['sub_category_1'].append(element.get("subCategory_1"))
            data_temp['wrong_word'].append(root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[int(begin):int(end)])
            data_temp['process'].append(element.get("process"))
            if "subCategory_2" in element.attrib:
                data_temp["sub_category_2"].append(element.get("subCategory_2"))
            else:
                data_temp["sub_category_2"].append("None")
            wrong_sentence,correct_sentence=wrong_correct_sentence(root,int(begin),int(end),element.get("correction"))
            data_temp["wrong_sentence"].append(wrong_sentence)
            data_temp["correct_sentence"].append(correct_sentence)

    df2=pd.DataFrame(data_temp)
    return df2

    

In [8]:
annotation_folders=list_all_folders("./all_data")
for folder in annotation_folders:
    all_items = os.listdir("./all_data/"+folder)
    xmi_files = [f for f in all_items if f.endswith('.xmi') and os.path.isfile(os.path.join("./all_data",folder, f))]
    for xmi_file in xmi_files:
        xmi_path=os.path.join("./all_data",folder,xmi_file)
    output= extract_data_from_XMI(xmi_path,xmi_file[:-4])
    if len(output)!=0:
        df=pd.concat([df,output],axis=0,ignore_index=True)
    


In [9]:
word_tokens_wrong=[]
word_tokens_correct=[]
error_index=[]
for index , row in df.iterrows():
    row['wrong_sentence']=' '.join(word_tokenize(row['wrong_sentence']))
    row['correct_sentence']=' '.join(word_tokenize(row['correct_sentence']))
    row['wrong_word']=' '.join(word_tokenize(row['wrong_word']))
    wrong_tokens=word_tokenize(row['wrong_sentence'])
    zeros=[0]*len(wrong_tokens)
    word_tokens_correct.append(word_tokenize(row['correct_sentence']))
    word_tokens_wrong.append(wrong_tokens)
    if len(row['wrong_word'].split(" "))>1:
        for i in range(0,len(wrong_tokens)):
            if row['wrong_word'].split(" ")[0]==wrong_tokens[i]:
                if row['wrong_word']==" ".join(wrong_tokens[i:i+len(row['wrong_word'].split(" "))]):
                    for index in range (i,i+len(row['wrong_word'].split(" "))):
                        zeros[index]=errors[row['main_category']]*10+sub_category_1[row['sub_category_1']]      
    else:
        for i in range(0,len(wrong_tokens)):
            if row['wrong_word']==wrong_tokens[i]:
                zeros[i]=errors[row['main_category']]*10+sub_category_1[row['sub_category_1']]
    if all(x == 0 for x in zeros):    
        for i in range(0,len(wrong_tokens)):
            under_line_list=wrong_tokens[i].split("_")
            if "_" in wrong_tokens[i]:
                if row['wrong_word']in under_line_list:
                    zeros[i]=errors[row['main_category']]*10+sub_category_1[row['sub_category_1']]
    if all(x == 0 for x in zeros):    
        for i in range(0,len(wrong_tokens)):
            if row['wrong_word'] in wrong_tokens[i]:
                zeros[i]=errors[row['main_category']]*10+sub_category_1[row['sub_category_1']]
    if all(x == 0 for x in zeros):    
        if len(row['wrong_word'].split(" "))>1:
            for i in range(0,len(wrong_tokens)):
                if (row['wrong_word'].split(" ")[0]==wrong_tokens[i] or row['wrong_word'].split(" ")[0] in wrong_tokens[i]) and i + len(row['wrong_word'].split(" "))<=len(wrong_tokens):
                    flag=True
                    for z in range(1,len(row['wrong_word'].split(" "))):
                        if row['wrong_word'].split(" ")[z]!=wrong_tokens[i+z] and row['wrong_word'].split(" ")[z] not in wrong_tokens[i+1]:
                            flag=False
                    if flag:
                        for index in range (i,i+len(row['wrong_word'].split(" "))):
                            zeros[index]=errors[row['main_category']]*10+sub_category_1[row['sub_category_1']]
    
    if all(x==0 for x in zeros):
        if row['main_category']=="Punctuation" and row['process']=="Omission":
            zeros[-1]=40
    if all(x==0 for x in zeros):
        print(row['wrong_sentence'])
        print(row)
        print()
                
        
    error_index.append(zeros)

بعد از ضهربرای تفریخ و کمک به پدربزرگم به باغ رفتم .
file_name                                       n_dictation_11_25.txt
USER_NUMBER                                                4011714026
begin                                                             279
end                                                               289
wrong_word                                                 بعد از ضهر
correct_word                                                 بعدازظهر
wrong_sentence      بعد از ضهربرای تفریخ و کمک به پدربزرگم به باغ ...
correct_sentence    بعدازظهربرای تفریخ و کمک به پدربزرگم به باغ رف...
process                                                   Replacement
main_category                                                    Form
sub_category_1                                               Boundary
sub_category_2                                                Merging
Name: 1494, dtype: object

این مشکل مخصوصا در کشورهای غربی ، مانند آمریکا و بعضی از کش . رهای اورپا دیده می

In [10]:
df['word_tokens_wrong']=word_tokens_wrong
df['word_tokens_correct']=word_tokens_correct
df['error_index']=error_index

In [11]:
s=0
for index , row in df.iterrows():
    if all(x == 0 for x in row['error_index']):
        s+=1
s

4

In [12]:
df.head()

,file_name,USER_NUMBER,begin,end,wrong_word,correct_word,wrong_sentence,correct_sentence,process,main_category,sub_category_1,sub_category_2,word_tokens_wrong,word_tokens_correct,error_index
0,n_dialog_11_3.txt,4011744020,140,145,داداش,داداش؟,سلام ، چطوری داداش الحمدلله خوبم انشاء الله,سلام ، چطوری داداش ؟ الحمدلله خوبم انشاء الله,Omission,Punctuation,None,None,"[سلام, ،, چطوری, داداش, الحمدلله, خوبم, انشاء,...","[سلام, ،, چطوری, داداش, ؟, الحمدلله, خوبم, انش...","[0, 0, 0, 40, 0, 0, 0, 0]"
1,n_dialog_11_3.txt,4011744020,181,185,خوبم,خوبم.,سلام ، چطوری داداش الحمدلله خوبم انشاء الله,سلام ، چطوری داداش الحمدلله خوبم . انشاء الله,Replacement,Punctuation,None,None,"[سلام, ،, چطوری, داداش, الحمدلله, خوبم, انشاء,...","[سلام, ،, چطوری, داداش, الحمدلله, خوبم, ., انش...","[0, 0, 0, 0, 0, 40, 0, 0]"
2,n_dialog_11_3.txt,4011744020,186,191,انشاء,ان شاء,سلام ، چطوری داداش الحمدلله خوبم انشاء الله,سلام ، چطوری داداش الحمدلله خوبم ان شاء الله,Replacement,Form,Boundary,Merging,"[سلام, ،, چطوری, داداش, الحمدلله, خوبم, انشاء,...","[سلام, ،, چطوری, داداش, الحمدلله, خوبم, ان, شا...","[0, 0, 0, 0, 0, 0, 31, 0]"
3,n_dialog_11_3.txt,4011744020,198,202,خوبی,خوبی؟,خوبی سلامتی انشاءالله شما هم خوب هستید,خوبی ؟ سلامتی انشاءالله شما هم خوب هستید,Omission,Punctuation,None,None,"[خوبی, سلامتی, انشاءالله, شما, هم, خوب, هستید]","[خوبی, ؟, سلامتی, انشاءالله, شما, هم, خوب, هستید]","[40, 0, 0, 0, 0, 0, 0]"
4,n_dialog_11_3.txt,4011744020,203,209,سلامتی,سلامتی؟,خوبی سلامتی انشاءالله شما هم خوب هستید,خوبی سلامتی ؟ انشاءالله شما هم خوب هستید,Omission,Punctuation,None,None,"[خوبی, سلامتی, انشاءالله, شما, هم, خوب, هستید]","[خوبی, سلامتی, ؟, انشاءالله, شما, هم, خوب, هستید]","[0, 40, 0, 0, 0, 0, 0]"


In [13]:
df.to_csv('data_temp.csv',index=True)

In [14]:
df.to_excel('data_temp.xlsx', index=False)